In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [94]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.block1 = nn.Sequential(

          nn.Conv2d(1, 4, 3),
          nn.BatchNorm2d(4),
          nn.ReLU(),

          nn.Conv2d(4, 8, 3),
          nn.BatchNorm2d(8),
          nn.ReLU(),

          nn.MaxPool2d(2, 2)

        )

        self.block2 = nn.Sequential(

          nn.Conv2d(8, 16, 3, padding = 1),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.Dropout2d(p=0.3),

          nn.Conv2d(16, 16, 3, padding = 1),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.Dropout2d(p=0.3),

          nn.Conv2d(16, 32, 3, padding = 1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.Dropout2d(p=0.3),

          nn.MaxPool2d(2, 2)

        )

        self.block3 = nn.Sequential(

          nn.Conv2d(32, 16, 3),
          nn.BatchNorm2d(16),
          nn.ReLU(),

          nn.AvgPool2d(2, 2),

          nn.Conv2d(16, 10, 2, bias=False, padding = 0)

        )

    def forward(self, x):

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)

        x = x.view(-1, 10)

        return F.log_softmax(x)

In [27]:
!pip install torchsummary

In [95]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              40
       BatchNorm2d-2            [-1, 4, 26, 26]               8
              ReLU-3            [-1, 4, 26, 26]               0
            Conv2d-4            [-1, 8, 24, 24]             296
       BatchNorm2d-5            [-1, 8, 24, 24]              16
              ReLU-6            [-1, 8, 24, 24]               0
         MaxPool2d-7            [-1, 8, 12, 12]               0
            Conv2d-8           [-1, 16, 12, 12]           1,168
       BatchNorm2d-9           [-1, 16, 12, 12]              32
             ReLU-10           [-1, 16, 12, 12]               0
        Dropout2d-11           [-1, 16, 12, 12]               0
           Conv2d-12           [-1, 16, 12, 12]           2,320
      BatchNorm2d-13           [-1, 16, 12, 12]              32
             ReLU-14           [-1, 16,

<ipython-input-94-a7ef1fe09e17>:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [96]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [72]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [97]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-94-a7ef1fe09e17>:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.06302039325237274 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.36it/s]



Test set: Average loss: 0.0848, Accuracy: 9726/10000 (97%)



loss=0.23149828612804413 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 43.20it/s]



Test set: Average loss: 0.0490, Accuracy: 9834/10000 (98%)



loss=0.02353212796151638 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 42.75it/s]


...


Test set: Average loss: 0.0233, Accuracy: 9932/10000 (99%)



loss=0.35760170221328735 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 40.26it/s]



Test set: Average loss: 0.0213, Accuracy: 9929/10000 (99%)



loss=0.0025749304331839085 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 42.73it/s]



Test set: Average loss: 0.0207, Accuracy: 9931/10000 (99%)

